# Predictive Used Car Prices

Utilizes pandas for data handling, sklearn for preprocessing (including one-hot encoding and scaling with **StandardScaler**), and **GridSearchCV** for hyperparameter tuning with **HistGradientBoostingRegressor** as the chosen machine learning model.

In [15]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import zipfile

# Load the training and test data
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# Preserve the 'id' column from the test data
test_ids = test_data['id']

# Convert categorical variables using one-hot encoding
train_data = pd.get_dummies(train_data)
test_data = pd.get_dummies(test_data)

# Align the train and test data to have the same columns
train_data, test_data = train_data.align(test_data, join='left', axis=1, fill_value=0)

# Remove the 'price' column from the test data
if 'price' in test_data.columns:
    test_data = test_data.drop(['price'], axis=1)

# Scale numerical features
scaler = StandardScaler()
numerical_columns = train_data.select_dtypes(include=['int64', 'float64']).columns.drop('price')
train_data[numerical_columns] = scaler.fit_transform(train_data[numerical_columns])
test_data[numerical_columns] = scaler.transform(test_data[numerical_columns])

# Define target and features
X = train_data.drop(['price'], axis=1)
y = train_data['price']

assert test_data.shape[0] == 36183, f"Expected 36183 rows in test data, but got {test_data.shape[0]}"


In [16]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the parameters for Grid Search
param_grid = {
    'learning_rate': [0.01, 0.1],
    'max_iter': [100, 200],
    'max_depth': [3, 5, 10],
    'min_samples_leaf': [20, 50]
}

# Gradient Boosting model
gb_model = HistGradientBoostingRegressor(random_state=42)

# Grid Search
grid_search = GridSearchCV(estimator=gb_model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

# Fit Grid Search to the data
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_score = np.sqrt(-grid_search.best_score_)
print(f'Best Parameters: {best_params}')
print(f'Best RMSE: {best_score}')


Fitting 3 folds for each of 24 candidates, totalling 72 fits


/Users/zoeyespinoza/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/zoeyespinoza/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/zoeyespinoza/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/zoeyespinoza/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/zoeyespinoza/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked

[CV] END learning_rate=0.01, max_depth=3, max_iter=100, min_samples_leaf=20; total time= 2.2min
[CV] END learning_rate=0.01, max_depth=3, max_iter=100, min_samples_leaf=50; total time= 2.2min
[CV] END learning_rate=0.01, max_depth=3, max_iter=100, min_samples_leaf=20; total time= 2.2min
[CV] END learning_rate=0.01, max_depth=3, max_iter=100, min_samples_leaf=20; total time= 2.3min
[CV] END learning_rate=0.01, max_depth=3, max_iter=100, min_samples_leaf=50; total time= 2.3min
[CV] END learning_rate=0.01, max_depth=3, max_iter=100, min_samples_leaf=50; total time= 2.3min
[CV] END learning_rate=0.01, max_depth=3, max_iter=200, min_samples_leaf=50; total time= 3.4min
[CV] END learning_rate=0.01, max_depth=3, max_iter=200, min_samples_leaf=50; total time= 3.4min
[CV] END learning_rate=0.01, max_depth=3, max_iter=200, min_samples_leaf=20; total time= 3.4min
[CV] END learning_rate=0.01, max_depth=3, max_iter=200, min_samples_leaf=20; total time= 3.5min
[CV] END learning_rate=0.01, max_depth=3

/Users/zoeyespinoza/anaconda3/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Users/zoeyespinoza/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


[CV] END learning_rate=0.1, max_depth=3, max_iter=200, min_samples_leaf=20; total time=  49.8s
[CV] END learning_rate=0.1, max_depth=3, max_iter=100, min_samples_leaf=50; total time= 1.3min
[CV] END learning_rate=0.1, max_depth=3, max_iter=100, min_samples_leaf=50; total time= 1.3min
[CV] END learning_rate=0.1, max_depth=3, max_iter=200, min_samples_leaf=20; total time= 1.0min
[CV] END learning_rate=0.1, max_depth=3, max_iter=200, min_samples_leaf=20; total time= 1.0min
[CV] END learning_rate=0.1, max_depth=3, max_iter=200, min_samples_leaf=50; total time=  59.2s
[CV] END learning_rate=0.1, max_depth=5, max_iter=100, min_samples_leaf=20; total time= 1.1min
[CV] END learning_rate=0.1, max_depth=3, max_iter=200, min_samples_leaf=50; total time= 1.3min
[CV] END learning_rate=0.1, max_depth=3, max_iter=200, min_samples_leaf=50; total time= 1.3min
[CV] END learning_rate=0.1, max_depth=5, max_iter=100, min_samples_leaf=20; total time= 1.2min
[CV] END learning_rate=0.1, max_depth=5, max_iter=

/Users/zoeyespinoza/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


[CV] END learning_rate=0.1, max_depth=5, max_iter=200, min_samples_leaf=20; total time= 1.2min
[CV] END learning_rate=0.01, max_depth=10, max_iter=200, min_samples_leaf=20; total time= 5.4min
[CV] END learning_rate=0.01, max_depth=10, max_iter=200, min_samples_leaf=20; total time= 5.4min
[CV] END learning_rate=0.1, max_depth=5, max_iter=200, min_samples_leaf=20; total time= 1.3min
[CV] END learning_rate=0.1, max_depth=5, max_iter=200, min_samples_leaf=50; total time= 1.2min
[CV] END learning_rate=0.01, max_depth=10, max_iter=200, min_samples_leaf=20; total time= 5.9min
[CV] END learning_rate=0.1, max_depth=5, max_iter=200, min_samples_leaf=50; total time= 1.4min
[CV] END learning_rate=0.1, max_depth=5, max_iter=200, min_samples_leaf=50; total time= 1.4min
[CV] END learning_rate=0.1, max_depth=10, max_iter=100, min_samples_leaf=20; total time= 1.3min
[CV] END learning_rate=0.1, max_depth=10, max_iter=100, min_samples_leaf=20; total time= 1.3min
[CV] END learning_rate=0.1, max_depth=10, 

In [17]:
# Train the Gradient Boosting model with the best parameters
best_gb_model = HistGradientBoostingRegressor(**best_params, random_state=42)
best_gb_model.fit(X_train, y_train)

y_pred_best_gb = best_gb_model.predict(X_val)

# Calculate RMSE for the best model
rmse_best_gb = np.sqrt(mean_squared_error(y_val, y_pred_best_gb))
print(f'RMSE (Best Gradient Boosting): {rmse_best_gb}')


RMSE (Best Gradient Boosting): 48139.49797036673


In [18]:
# Make predictions on the test set
test_predictions = best_gb_model.predict(test_data)

submission = pd.DataFrame({
    'id': test_ids,  
    'price': test_predictions
})

assert submission.shape[0] == 36183, f"Expected 36183 rows in submission, but got {submission.shape[0]}"

# Save the submission file
submission.to_csv('submission.csv', index=False)
print('Submission file saved as submission.csv')

with zipfile.ZipFile('submission.zip', 'w') as zf:
    zf.write('submission.csv')

print('Submission file zipped as submission.zip')


Submission file saved as submission.csv
Submission file zipped as submission.zip
